In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.document_loaders import UnstructuredPDFLoader
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from coexpert.params import OPENAI_API_KEY


In [8]:
path = './raw_data'
loader = PyPDFDirectoryLoader(path)
documents = loader.load()
print(documents)

# split the documents in small chunks
# Change the chunk_size and chunk_overlap as needed
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=100)
all_splits = text_splitter.split_documents(documents)

pages = all_splits

# Replace book.pdf with any pdf of your choice
# loader = UnstructuredPDFLoader("../../raw_data/_51113879-001A_NG4_Tachy_CRT-D_PTM_en_S.pdf")
# pages = loader.load_and_split()


[Document(page_content='PHYSICIAN’S TECHNICAL MANUAL\nRESONATE™HFICD,\nRESONATE™ELICD,\nPERCIVA™HFICD,\nPERCIVA™ICD,\nVIGILANT™ELICD,\nMOMENTUM™ELICD\nIMPLANTABLE CARDIOVERTER DEFIBRILLATOR\nREFD520,D521,D532,D533,D420,D421,D432,D433,D500,D501,D512,D513,D400,D401,\nD412,D413,D220,D221,D232,D233,D120,D121\nCAUTION: Federallaw(USA)restricts\nthisdevicetosalebyoronthe\norderofaphysician trainedor\nexperiencedindeviceimplantand\nfollow-up procedures.', metadata={'source': 'raw_data/_51114084-001A_NG4 Tachy ICD_PTM_en_S.pdf', 'page': 0}), Document(page_content='', metadata={'source': 'raw_data/_51114084-001A_NG4 Tachy ICD_PTM_en_S.pdf', 'page': 1}), Document(page_content='TableofContents\nTrademarkStatement ..........................................................................................................................1\nAdditional Information .........................................................................................................................1\nDeviceDescriptio

In [9]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
docsearch = Chroma.from_documents(pages, embeddings).as_retriever()


Using embedded DuckDB without persistence: data will be transient


In [10]:
# Choose any query of your choice
query = "Give me the dimensions of defibrilators?"
# query = "Give me position of kamasutra"
docs = docsearch.get_relevant_documents(query)
# chain = load_qa_chain(OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff")
chain = load_qa_chain(ChatOpenAI(
    temperature=0, openai_api_key=OPENAI_API_KEY), chain_type="stuff")
output = chain.run(input_documents=docs, question=query)
print(output)


Here are the dimensions of the defibrillators mentioned in the provided context:

RESONATE HFICDs:
- D520(VR): 5.37x7.79x0.99 cm
- D521(DR): 5.37x7.79x0.99 cm
- D532(VR): 5.37x7.36x0.99 cm
- D533(DR): 5.37x7.68x0.99 cm

RESONATE Extended Longevity (EL)ICDs:
- D420(VR): 5.37x7.79x0.99 cm
- D421(DR): 5.37x7.79x0.99 cm
- D432(VR): 5.37x7.36x0.99 cm
- D433(DR): 5.37x7.68x0.99 cm

PERCIVAHFICDs:
- D500(VR): 5.23x7.14x0.99 cm
- D501(DR): 5.23x7.14x0.99 cm
- D512(VR): 5.23x6.71x0.99 cm
- D513(DR): 5.23x7.03x0.99 cm

RESONATE HFCRT-Ds:
- G524: 5.37x8.18x0.99 cm
- G525: 5.37x8.08x0.99 cm
- G526: 5.37x8.08x0.99 cm
- G528: 5.37x8.08x0.99 cm
